# Segmenting and Clustering Neighborhoods in Toronto City

### 1. Download and Explore Dataset

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Removing all Boroughs with no names

In [2]:
df = df[df.Borough != 'Not assigned']

In [3]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


#### Assigning Borough names to Neighbourhoods with no name

In [4]:
df = df.replace('Not assigned',df.Borough)

In [5]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


#### Grouping all areas with similar Postal codes

In [6]:
df = df.groupby(['PostalCode','Borough']).Neighbourhood.unique().reset_index()

In [7]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


##### Getting the size of the dataset

In [8]:
df.shape

(103, 3)

#### Importing Latitude & Longitude from CSV

In [9]:
url = r'/Users/happy/Downloads/Geospatial_Coordinates.csv'
df1=pd.read_csv(url)

In [10]:
df1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging the CSV with our dataframe

In [11]:
neighborhoods = pd.merge(df,
                 df1[['PostalCode', 'Latitude', 'Longitude']],
                 on='PostalCode')

Quickly examine the resulting dataframe.

In [12]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


#### Use geopy library to get the latitude and longitude values of Toronto City

In [13]:
address = 'Toronto City, YTZ'

geolocator = Nominatim(user_agent="ytz_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.62805705, -79.3979911558998.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's simplify the above map and segment and cluster only the neighborhoods in North York. So let's slice the original dataframe and create a new dataframe of the North York data.

In [15]:
ny_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
ny_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,[Hillcrest Village],43.803762,-79.363452
1,M2J,North York,"[Fairview, Henry Farm, Oriole]",43.778517,-79.346556
2,M2K,North York,[Bayview Village],43.786947,-79.385975
3,M2L,North York,"[Silver Hills, York Mills]",43.757490,-79.374714
4,M2M,North York,"[Newtonbrook, Willowdale]",43.789053,-79.408493


Let's get the geographical coordinates of North York data

In [16]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ytz_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [17]:
# create map of North York using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ny_data['Latitude'], ny_data['Longitude'], ny_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'Q0B5XTSNTRQ4IIWAETBLTDUZYBXSQ2S4B4RRBKYNVDG0R5U2' # your Foursquare ID
CLIENT_SECRET = 'YGZXBU1DXAAJ4T0ZNH5QFDJE3ZXSZYAT4TXFNCS1MQMUPYA1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q0B5XTSNTRQ4IIWAETBLTDUZYBXSQ2S4B4RRBKYNVDG0R5U2
CLIENT_SECRET:YGZXBU1DXAAJ4T0ZNH5QFDJE3ZXSZYAT4TXFNCS1MQMUPYA1


#### Let's explore the first neighborhood in our dataframe.

In [19]:
ny_data.loc[0, 'Neighbourhood']

array(['Hillcrest Village'], dtype=object)

In [20]:
neighborhood_latitude = ny_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ny_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = ny_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Hillcrest Village'] are 43.8037622, -79.3634517.


#### Now, let's get the top 100 venues that are in Hillcrest Village within a radius of 500 meters.

In [21]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=Q0B5XTSNTRQ4IIWAETBLTDUZYBXSQ2S4B4RRBKYNVDG0R5U2&client_secret=YGZXBU1DXAAJ4T0ZNH5QFDJE3ZXSZYAT4TXFNCS1MQMUPYA1&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'

Send the GET request and examine the resutls

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8c9e0c923935002848ea92'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'Toronto

Let's borrow the **get_category_type** function from the Foursquare lab.

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in North York

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *ny_venues*.

In [27]:
ny_venues = getNearbyVenues(names=ny_data['Neighbourhood'],
                                   latitudes=ny_data['Latitude'],
                                   longitudes=ny_data['Longitude']
                                  )

['Hillcrest Village']
['Fairview' 'Henry Farm' 'Oriole']
['Bayview Village']
['Silver Hills' 'York Mills']
['Newtonbrook' 'Willowdale']
['Willowdale South']
['York Mills West']
['Willowdale West']
['Parkwoods']
['Don Mills North']
['Flemingdon Park' 'Don Mills South']
['Bathurst Manor' 'Downsview North' 'Wilson Heights']
['Northwood Park' 'York University']
['CFB Toronto' 'Downsview East']
['Downsview West']
['Downsview Central']
['Downsview Northwest']
['Victoria Village']
['Bedford Park' 'Lawrence Manor East']
['Lawrence Heights' 'Lawrence Manor']
['Glencairn']
['Downsview' 'North Park' 'Upwood Park']
['Humber Summit']
['Emery' 'Humberlea']


#### Let's check the size of the resulting dataframe

In [28]:
print(ny_venues.shape)
ny_venues.head()

(241, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,[Hillcrest Village],43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,[Hillcrest Village],43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,[Hillcrest Village],43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,[Hillcrest Village],43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"[Fairview, Henry Farm, Oriole]",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


Let's check how many venues were returned for each neighborhood

In [29]:
ny_venues['Neighbourhood'].groupby([ny_venues['Venue'], ny_venues['Venue Category']]).count().reset_index()

,Venue,Venue Category,Neighbourhood
0,649 Variety,Convenience Store,1
1,AY Jackson Pool,Pool,1
2,Aburi Room,Sushi Restaurant,1
3,Ajisen Ramen 味千ラーメン,Ramen Restaurant,1
4,Ancaster Park,Park,1
5,Apple Fairview,Electronics Store,1
6,Ardene,Women's Store,1
7,Ardene Shoes Outlet,Accessories Store,1
8,Aroma Espresso Bar,Café,2
9,Aroma Espresso Bar,Coffee Shop,1


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} unique categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 105 unique categories.


### Analyze Each Neighborhood

In [31]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighbourhood'] = ny_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,[Hillcrest Village],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,[Hillcrest Village],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,[Hillcrest Village],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,[Hillcrest Village],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[Fairview, Henry Farm, Oriole]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


And let's examine the new dataframe size.

In [32]:
ny_onehot.shape

(241, 106)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
ny_grouped = ny_onehot.groupby([ny_onehot['Neighbourhood'].apply(tuple)]).mean().reset_index()
ny_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"(Bathurst Manor, Downsview North, Wilson Heights)",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,0.0,0.055556,0.000000,0.000000,0.0,0.000000,0.00,0.055556,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.055556,0.055556,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,"(Bayview Village,)",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"(Bedford Park, Lawrence Manor East)",0.000000,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.04,0.000000,0.00,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.040000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.040000,0.040000,0.000000,0.0,0.00,0.000000,0.000000,0.04,0.000000,0.080000,0.000000,0.040000,0.040000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.

And let's examine the new dataframe size.

In [34]:
ny_grouped.shape

(23, 106)

In [35]:
ny_grouped['Neighbourhood'] = ny_grouped['Neighbourhood'].apply(np.str)

#### Let's print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in ny_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----('Bathurst Manor', 'Downsview North', 'Wilson Heights')----
                  venue  freq
0           Coffee Shop  0.11
1      Sushi Restaurant  0.06
2    Frozen Yogurt Shop  0.06
3  Fast Food Restaurant  0.06
4        Sandwich Place  0.06


----('Bayview Village',)----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----('Bedford Park', 'Lawrence Manor East')----
                venue  freq
0         Pizza Place  0.08
1  Italian Restaurant  0.08
2         Coffee Shop  0.08
3    Sushi Restaurant  0.08
4    Greek Restaurant  0.04


----('CFB Toronto', 'Downsview East')----
                           venue  freq
0                        Airport   0.5
1                           Park   0.5
2                       Pharmacy   0.0
3  Paper / Office Supplies Store   0.0
4                  Movie Theater   0.0


----('Don Mills North',)----
                  venue 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"('Bathurst Manor', 'Downsview North', 'Wilson ...",Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Diner,Restaurant,Sandwich Place
1,"('Bayview Village',)",Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
2,"('Bedford Park', 'Lawrence Manor East')",Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Greek Restaurant,Sandwich Place,Cupcake Shop,Liquor Store,Café,Butcher
3,"('CFB Toronto', 'Downsview East')",Park,Airport,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,"('Don Mills North',)",Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [39]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 0, 0, 1, 0, 4, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = ny_data

In [41]:
ny_merged=pd.concat((ny_merged, neighborhoods_venues_sorted), axis=1, join='inner')

In [42]:
ny_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,[Hillcrest Village],43.803762,-79.363452,0,"('Bathurst Manor', 'Downsview North', 'Wilson ...",Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Diner,Restaurant,Sandwich Place
1,M2J,North York,"[Fairview, Henry Farm, Oriole]",43.778517,-79.346556,0,"('Bayview Village',)",Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
2,M2K,North York,[Bayview Village],43.786947,-79.385975,0,"('Bedford Park', 'Lawrence Manor East')",Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Greek Restaurant,Sandwich Place,Cupcake Shop,Liquor Store,Café,Butcher
3,M2L,North York,"[Silver Hills, York Mills]",43.757490,-79.374714,2,"('CFB Toronto', 'Downsview East')",Park,Airport,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,M2M,North York,"[Newtonbrook, Willowdale]",43.789053,-79.408493,0,"('Don Mills North',)",Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop


Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [44]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,Neighbourhood,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,[Hillcrest Village],"('Bathurst Manor', 'Downsview North', 'Wilson ...",Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Diner,Restaurant,Sandwich Place
1,North York,0,"[Fairview, Henry Farm, Oriole]","('Bayview Village',)",Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
2,North York,0,[Bayview Village],"('Bedford Park', 'Lawrence Manor East')",Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Greek Restaurant,Sandwich Place,Cupcake Shop,Liquor Store,Café,Butcher
4,North York,0,"[Newtonbrook, Willowdale]","('Don Mills North',)",Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Discount Store,Convenience Store,Cosmetics Shop,Cupcake Shop
5,North York,0,[Willowdale South],"('Downsview', 'North Park', 'Upwood Park')",Construction & Landscaping,Bakery,Basketball Court,Park,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
7,North York,0,[Willowdale West],"('Downsview Northwest',)",Liquor Store,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Women's Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
10,North York,0,"[Flemingdon Park, Don Mills South]","('Fairview', 'Henry Farm', 'Oriole')",Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Japanese Restaurant,Food Court,Women's Store,Asian Restaurant,Toy / Game Store,Baseball Field
11,North York,0,"[Bathurst Manor, Downsview North, Wilson Heights]","('Flemingdon Park', 'Don Mills South')",Coffee Shop,Asian Restaurant,Gym,Beer Store,Chinese Restaurant,Sporting Goods Shop,Clothing Store,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant
12,North York,0,"[Northwood Park, York University]","('Glencairn',)",Pizza Place,Park,Asian Restaurant,Pub,Japanese Restaurant,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
13,North York,0,"[CFB Toronto, Downsview East]","('Hillcrest Village',)",Golf Course,Mediterranean Restaurant,Pool,Dog Run,Fried Chicken Joint,Dim Sum Restaurant,Construction & Landscaping,Convenience Store,Furniture / Home Store,Cosmetics Shop


#### Cluster 2

In [45]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,Neighbourhood,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,1,[York Mills West],"('Downsview Central',)",Food Truck,Baseball Field,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
9,North York,1,[Don Mills North],"('Emery', 'Humberlea')",Paper / Office Supplies Store,Baseball Field,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
